# 1 - install dependencies

In [4]:
import numpy as np
import cv2 
import os
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

2022-04-14 15:12:05.233640: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mina/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-14 15:12:05.233658: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [28]:

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

labels_dic = {
    "aleff": 0, #أ
    "bb": 1,    #ب
    "taa": 2,   #ت
    "thaa": 3,  #ث
    "jeem": 4,  #ج
    "haa": 5,   #ح
    "khaa": 6,  #خ
    "dal": 7,   #د
    "thal": 8,  #ذ
    "ra": 9,    #ر
    "zay": 10,  #ز
    "seen": 11, #س
    "sheen": 12,#ش
    "saad": 13, #ص
    "dhad": 14, #ض
    "ta": 15,   #ط
    "dha": 16,  #ظ
    "ain": 17,  #ع
    "ghain": 18,#غ
    "fa": 19,   #ف
    "gaaf": 20, #ق
    "kaaf": 21, #ك
    "laam": 22, #ل
    "meem": 23, #م
    "nun": 24,  #ن
    "ha": 25,   #هـ
    "waw": 26,  #و
    "ya": 27,   #ئ
    "toot": 28, #ة
    "al": 29,   #ال
    "la": 30,   #لا
    "yaa": 31   #ي
}

    
def draw_styled_landmarks(image,results):
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
  
                
def extract_keypoints(results):
    if results.right_hand_landmarks:
        return np.array([ [res.x,res.y] for res in results.right_hand_landmarks.landmark ]).flatten()
    else :
        return np.zeros(42,dtype=np.float32)



# holistic model process image and return the results as keypoints
def mediapipe_detection(image,model):
    image  = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image  = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results
    
actions = [data for data in labels_dic]

# 4 - build the model

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Input,Dropout
from tensorflow.keras.models import Model

best_model_file_name = os.path.join("weights","V1.h5")


def first_model():
    input_layer = Input(shape=(42,))
    layer = Dense(128,activation="relu")(input_layer)
    layer = Dense(256,activation="relu")(layer)
    layer = Dense(128,activation="relu")(layer)
    layer = Dense(len(labels_dic),activation="softmax")(layer)

    model = Model(inputs=input_layer,outputs=layer)
    model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = first_model()
model.summary()

model.load_weights(best_model_file_name)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 42)]              0         
                                                                 
 dense (Dense)               (None, 128)               5504      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 32)                4128      
                                                                 
Total params: 75,552
Trainable params: 75,552
Non-trainable params: 0
_________________________________________________________________


2022-04-14 15:12:27.617016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-14 15:12:27.618749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mina/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-14 15:12:27.618957: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mina/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-04-14 15:12:27.619135: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublas

# 5 - test in real time

In [29]:
import numpy as np
import cv2 
import os
import mediapipe as mp
import tensorflow as tf 
from tensorflow import keras
import matplotlib.pyplot as plt
import time
import pandas as pd

In [30]:

def view_probability(res, actions, image):
    #output_frame = input_frame.copy()
    
    output_frame = image
    height,width,_ = image.shape
    
    if(type(res)==list or type(res)==np.ndarray):
        max_prob_index = np.argmax(res)
        max_prob = res[max_prob_index]
        text = f'{actions[max_prob_index]} - {max_prob:.3f}'
    else:
        text = "not signing"
    
    
    cv2.rectangle(output_frame, (0,0), (width, 40), (0,255,0), -1)
    cv2.putText(output_frame, text, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame



def view_sentence(sentence,image):
    # output_frame = image.copy()
    output_frame = image
    height,width,_ = image.shape
    cv2.rectangle(output_frame, (0,height-40), (width, height), (255, 0, 0), -1)
    cv2.putText(output_frame, ' '.join(sentence), (10,height-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame


def compare_frames(prev_frame,current_frame,threshold):
    return True
    if(type(prev_frame) == np.ndarray ):
        diff = cv2.absdiff(prev_frame, current_frame)
        s = diff.sum()
        if(s > threshold):
            return True
        
    return False
    
def evaluate_list_comparisons(s):
    return True
    if(sum(s)>=2):
        return True
    return False

In [47]:
# 1. New detection variables

holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)


sequence = [np.zeros(62)]
sentence = []
predictions = []
threshold = 0.5

res = None

cap = cv2.VideoCapture(0)
# Set mediapipe model 

last_comparisons = []

prev_frame = None
while cap.isOpened():

    ret, frame = cap.read()
    if(not ret):break
        
    image, results = mediapipe_detection(frame, holistic)
    keypoints = extract_keypoints(results)
    res = model.predict(np.array([keypoints]))[0]
    draw_styled_landmarks(image,results)
    
    cv2.imshow('OpenCV Feed', image)

    image = view_probability(res, actions, image)
    

    
    cv2.imshow('OpenCV Feed', image)

    # Break
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

(640, 840, 3)

In [39]:
cap.release()

In [40]:
cv2.destroyAllWindows()

In [26]:
old = np.array([5,2,2,2,2,2])
test = np.array([5,2,3,0,5,1])


In [30]:
test&test

array([5, 2, 3, 0, 5, 1])

In [32]:
for ind,n in enumerate(test):
    if test[ind]==0:test[ind] = old[ind]

In [33]:
test

array([5, 2, 3, 2, 5, 1])